In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from data_loader import data_label_split
from data_loader import generate_data_set

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.utils.data as D 
import torch.nn as nn

In [ ]:
drop_NA_data=pd.read_csv("moa_data_drop_NA.csv", index_col=0)

In [4]:
drop_NA_data['compound'].value_counts()

DMSO     244072
taxol     56763
Name: compound, dtype: int64

In [ ]:
# takes the df with "compound" columns
class dmso_taxol_ProfileBag(D.Dataset):
    def __init__(self, df, size, bag_mean_size, bag_std_size, perc, treatment, control):
        self.df = df
        self.size = size
        self.bag_mean_size = bag_mean_size
        self.bag_std_size = bag_std_size
        self.perc = perc
        self.treatment = treatment
        self.control = control
        self.bag_size = np.random.normal(bag_mean_size, bag_std_size, size).astype(int)
        
    def __getitem__(self, index):
        treatment_data = self.df[self.df["compound"] == self.treatment]
        control_data = self.df[self.df["compound"] == self.control]
        treatment_size = (self.bag_size * self.perc).astype(int)
        control_size = self.bag_size - treatment_size
        
        treatment_data
        
        _data = self.df[(self.df["TableNumber"]==table) & (self.df["ImageNumber"]==image)]
        label = list(_data[["compound","concentration","moa"]].reset_index(drop=True).iloc[0])
        return torch.tensor(np.array(_data.drop(["Image_Metadata_Compound","Image_Metadata_Concentration",
                                         "compound","concentration","moa"], axis = 1))).float(), label
        
    def __len__(self):
        return len(self.meta)

In [ ]:
return bags with different size (mean, std) but same proportion (perc)

In [ ]:
treatment_data = data[data["compound"] == treatment]
control_data = data[data["compound"] == control]
num_treat = int(size * perc)
return treatment_data.sample(num_treat).append(control_data.sample(size - num_treat)).sample(frac = 1).reset_index(drop=True)

In [16]:
bag_size = np.random.normal(5, 1, 10).astype(int)

In [20]:
bag_size-(bag_size*0.3).astype(int)

array([3, 4, 3, 3, 4, 4, 3, 4, 3, 4])